<a href="https://colab.research.google.com/github/m-manuelmussa/Desenvolvimento-de-modelos-de-machine-learning-para-predizer-novos-inibidores-da-TK-HER2/blob/main/2_Pr%C3%A9_processamento_de_dados_%26_C%C3%A1lculo_de_Features_de_Lipinski_%26_PubChemFP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ***Desenvolvimento de modelos de machine learning baseados em QSAR-3D para a predição de novos candidatos a fármacos TKI-HER2 para o tratamento de câncer da mama***

---



***Autor: Micliete_Mussa***

### ***2. Machine Learning & QSAR-3D***

### ***2.1. Pré-processamento de dados***

### **Pipeline**
1. Configuração do ambiente de trabalho
2. Carregamento dos dados em .csv
3. Conversão de SMILES em Mol
4. Remoção de sais
5. Padronização de tautômeros
6. Remoção de moléculas não Kekulizadas
7. Remoção de duplicatas
8. Conversão de IC50 em pIC50
9. Binarização de pIC50 em activo/Inactivo

### ***1. Configuração do ambiente de trabalho***

In [1]:
#Instalação de frameworks necessários
!pip install rdkit

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.9/34.9 MB 19.1 MB/s eta 0:00:00


In [3]:
#Instalação de frameworks necessários
import pandas as pd
import numpy as np
from rdkit import Chem
from rdkit.Chem import SaltRemover, MolStandardize
from rdkit.Chem.MolStandardize import rdMolStandardize
from rdkit.Chem import Descriptors, Lipinski, AllChem
from rdkit.Chem import rdMolDescriptors
from google.colab import files

### ***2. Carregamento dos dados em .csv***

In [5]:
df = pd.read_csv("Dataset_final.csv", sep=";")  # Deve ter colunas 'SMILES' e 'IC50'

### ***3. Conversão de SMILES em Mol***

In [6]:
df['Mol'] = df['SMILES'].apply(lambda x: Chem.MolFromSmiles(x))
df = df[df['Mol'].notnull()].reset_index(drop=True)

### ***4. Remoção de sais***

In [7]:
remover = SaltRemover.SaltRemover()
def remover_sais(mol):
    try:
        mol = remover.StripMol(mol, dontRemoveEverything=True)
        return mol
    except:
        return None
df['Mol'] = df['Mol'].apply(remover_sais)
df = df[df['Mol'].notnull()].reset_index(drop=True)

### ***5. Padronização de tautômeros***

In [8]:
tautomer_enumerator = rdMolStandardize.TautomerEnumerator()
df['Mol'] = df['Mol'].apply(lambda mol: tautomer_enumerator.Canonicalize(mol))

[09:12:49] Tautomer enumeration stopped at 171 tautomers: max transforms reached
[09:12:50] Tautomer enumeration stopped at 171 tautomers: max transforms reached
[09:12:51] Tautomer enumeration stopped at 171 tautomers: max transforms reached
[09:12:51] Tautomer enumeration stopped at 156 tautomers: max transforms reached
[09:12:52] Tautomer enumeration stopped at 165 tautomers: max transforms reached
[09:12:53] Tautomer enumeration stopped at 171 tautomers: max transforms reached
[09:12:54] Tautomer enumeration stopped at 171 tautomers: max transforms reached
[09:12:54] Tautomer enumeration stopped at 171 tautomers: max transforms reached
[09:12:55] Tautomer enumeration stopped at 183 tautomers: max transforms reached
[09:12:56] Tautomer enumeration stopped at 195 tautomers: max transforms reached
[09:12:57] Tautomer enumeration stopped at 171 tautomers: max transforms reached
[09:12:59] Tautomer enumeration stopped at 156 tautomers: max transforms reached
[09:13:06] Can't kekulize mo

### ***6. Remoção de moléculas não Kekulizadas***

In [9]:
def mol_valido(mol):
    try:
        Chem.Kekulize(mol, clearAromaticFlags=True)
        return True
    except:
        return False

df['Kekuliza'] = df['Mol'].apply(mol_valido)
df = df[df['Kekuliza']].reset_index(drop=True)
df.drop(columns=['Kekuliza'], inplace=True)

### ***7. Remoção de duplicatas***

In [10]:
df['Canonical_SMILES'] = df['Mol'].apply(lambda mol: Chem.MolToSmiles(mol))
df = df.drop_duplicates(subset='Canonical_SMILES').reset_index(drop=True)

### ***8. Conversão de IC50 em pIC50***

In [11]:
# IC50 deve estar em nM (nanomolar)
def converter_para_pIC50(ic50_nM):
    try:
        if ic50_nM > 0:
            return -np.log10(ic50_nM * 1e-9)
        else:
            return np.nan
    except:
        return np.nan

df['pIC50'] = df['IC50_nM'].apply(converter_para_pIC50)
df = df[df['pIC50'].notnull()].reset_index(drop=True)


### ***9. Categorização de valores de pIC50***

In [12]:
# Definir limiar (pIC50 >= 6.0 → ativo (1); senão → inativo (0))
limiar = 6.0

# Binarizar em 0 e 1
df['Classe'] = df['pIC50'].apply(lambda x: 1 if x >= limiar else 0)

### ***10. Dados pré-processados***

In [13]:
df['Name'] = df.index.astype(str)
df_final = df[['Canonical_SMILES', 'IC50_nM', 'pIC50', 'Classe']]
display(df_final.head())

,Canonical_SMILES,IC50_nM,pIC50,Classe
0,C=C1C=C(C)NC1=CC1=C(O)NC2=NC=NC(NC3=CC=C(F)C(C...,300.0,6.522879,1
1,C=C1C=C(C(=O)N2CCOCC2)NC1=CC1=C(O)NC2=NC=NC(NC...,400.0,6.397940,1
2,C=C1C=C(C(=O)N2CCOCC2)NC1=CC1=C(O)NC2=NC=NC(NC...,100.0,7.000000,1
3,C=C1C=C(C(=O)N2CCOCC2)NC1=CC1=C(O)NC2=NC=NC(NC...,5000.0,5.301030,0
4,C=C1C=C(C(=O)NCCN2CCOCC2)NC1=CC1=C(O)NC2=NC=NC...,100.0,7.000000,1


### ***2.2. Cálculo de descritores e Fingerprints***

## **Pipeline**
1. Seleccionar descritores de Lipinski e Fingerprints de PubChem
2. Calcular descritores de Lipinski e PubChemFP para todos SMILES
3. Concatenar com dados pré-processados
4. Salvar os dados para EDA

### ***1. Seleccionar descritores de Lipinski e Fingerprints de PubChem***

In [14]:
def calcular_descritores_fp(smiles):
    mol = Chem.MolFromSmiles(smiles)

    # === Descritores de Lipinski ===
    mol_logp = Descriptors.MolLogP(mol)
    mol_wt = Descriptors.MolWt(mol)
    hbd = Lipinski.NumHDonors(mol)
    hba = Lipinski.NumHAcceptors(mol)

    # === Fingerprint tipo PubChem ===
    fp = Chem.RDKFingerprint(mol, fpSize=881)
    fp_bits = list(fp)

    return [mol_logp, mol_wt, hbd, hba] + fp_bits

### ***2. Calcular descritores de Lipinski e PubChemFP para todos SMILES***

In [15]:
# Nomes das colunas
colunas_descritores = ['MolLogP', 'MolWt', 'nHBDon', 'nHBAcc']
colunas_fingerprints = [f'PubChemFP_{i}' for i in range(881)]

# Calcular para todos os SMILES
dados = df['SMILES'].apply(calcular_descritores_fp)

df_descritores_fp = pd.DataFrame(dados.tolist(), columns=colunas_descritores + colunas_fingerprints)

### ***3. Concatenar com dados pré-processados***

In [16]:
# Juntar com informações originais
df_final = pd.concat([df.reset_index(drop=True), df_descritores_fp.reset_index(drop=True)], axis=1)

# Ver resultado
print(df_final.shape)
df_final.head()

(2694, 893)


,ChEMBL_ID,SMILES,IC50_nM,Mol,Canonical_SMILES,pIC50,Classe,Name,MolLogP,MolWt,...,PubChemFP_871,PubChemFP_872,PubChemFP_873,PubChemFP_874,PubChemFP_875,PubChemFP_876,PubChemFP_877,PubChemFP_878,PubChemFP_879,PubChemFP_880
0,CHEMBL68920,Cc1cc(C)c(/C=C2\C(=O)Nc3ncnc(Nc4ccc(F)c(Cl)c4)...,300.0,<rdkit.Chem.rdchem.Mol object at 0x7b08b8c7b5a0>,C=C1C=C(C)NC1=CC1=C(O)NC2=NC=NC(NC3=CC=C(F)C(C...,6.522879,1,0,4.45034,383.814,...,1,1,1,0,1,1,1,1,1,1
1,CHEMBL69960,Cc1cc(C(=O)N2CCOCC2)[nH]c1/C=C1\C(=O)Nc2ncnc(N...,400.0,<rdkit.Chem.rdchem.Mol object at 0x7b08b8c7b8b0>,C=C1C=C(C(=O)N2CCOCC2)NC1=CC1=C(O)NC2=NC=NC(NC...,6.397940,1,1,3.61432,482.903,...,1,1,1,0,1,1,1,1,1,1
2,CHEMBL67057,Cc1cc(C(=O)N2CCOCC2)[nH]c1/C=C1\C(=O)Nc2ncnc(N...,100.0,<rdkit.Chem.rdchem.Mol object at 0x7b08b8c7ac70>,C=C1C=C(C(=O)N2CCOCC2)NC1=CC1=C(O)NC2=NC=NC(NC...,7.000000,1,2,4.82482,559.630,...,1,1,1,1,1,1,1,1,1,1
3,CHEMBL65848,Cc1cc(C(=O)N2CCOCC2)[nH]c1/C=C1\C(=O)Nc2ncnc(N...,5000.0,<rdkit.Chem.rdchem.Mol object at 0x7b08b8f46c00>,C=C1C=C(C(=O)N2CCOCC2)NC1=CC1=C(O)NC2=NC=NC(NC...,5.301030,0,3,3.31052,470.533,...,1,1,1,0,1,1,1,1,1,1
4,CHEMBL69629,Cc1cc(C(=O)NCCN2CCOCC2)[nH]c1/C=C1\C(=O)Nc2ncn...,100.0,<rdkit.Chem.rdchem.Mol object at 0x7b08b8f46b90>,C=C1C=C(C(=O)NCCN2CCOCC2)NC1=CC1=C(O)NC2=NC=NC...,7.000000,1,4,3.20392,525.972,...,1,1,1,0,1,1,1,1,1,1


### ***4. Salvar os dados para EDA***

In [17]:
df.to_csv('Dados para EDA.csv', index=False, encoding='utf-8')

### ***FIM***

### ***Referências***
1. [Numpy](https://numpy.org/)

2. [Pandas](https://pandas.pydata.org/)

3. [RDKit](https://www.rdkit.org/)
